In [ ]:
import datetime
import json
from functools import reduce

In [ ]:
# Criar novos registros e identificar a data que o registro foi feito, qual tipo de movimentação, valor.

def new_file(title: str="financas"):
  try:
    open(f'{title}_historico.json','x')
    txt = "Arquivo de histórico criado com sucesso.\n\n"

  except:
    txt = "O arquivo de histórico já existe.\n\n"

  finally:
    print(txt)
    chat_pagina1()

def update_entry(): #gaby
    historico = get_history()  # Chamando o get_history onde pego os dados do nosso json

    entry_id = input("Informe o ID da movimentação que deseja atualizar: ")  # Informo o ID para atualizar
    if entry_id not in historico:
        print("ID não encontrado.")
        # chat_pagina2()

    print("Informações atuais:", historico[entry_id])

    tipo_movimentacao = input("Informe o tipo da movimentação (receita, despesa, investimento): ")
    if tipo_movimentacao not in ["receita", "despesa", "investimento"]:
        print("Tipo de movimentação inválido.")
        chat_pagina2()

    novo_valor = input("Informe o novo valor: ")  # Insiro a quantia
    try:
        novo_valor = float(novo_valor)

        if tipo_movimentacao == "receita":
          if "juros" in historico[entry_id].keys():
            del historico[entry_id]["juros"]

        if tipo_movimentacao == "despesa":
          novo_valor *= -1
          if "juros" in historico[entry_id].keys():
            del historico[entry_id]["juros"]

        if tipo_movimentacao == "investimento":
          taxa = float(input("Digite a taxa diária do investimento, em porcentagem: "))
          historico[entry_id]['juros'] = taxa

        historico[entry_id]["movimentacao"] = tipo_movimentacao  # Atualizando o tipo de movimentação
        historico[entry_id]["valor"] = novo_valor  # Colocando novo valor

        save_json_file('financas_historico.json', historico)  # Salva de novo
        print("Movimentação atualizada com sucesso.")
        chat_pagina2()

    except ValueError:
        print("Valor inválido.") #gaby


def delete_entry(): # Daniel (a ideia era que a função get_report() substituísse o trecho do 'with' até o 'print(historico_filtrado)', mas estava dando erro).
  with open('financas_historico.json','r') as file:
    conteudo = file.read()
    if len(conteudo) == 0:
      historico = {}
    else:
      historico = json.loads(conteudo)
  print("""Abaixo está o seu histórico de lançamentos.
  Copie o id correspondente caso deseje atualizar/deletar algum registro (ex. 202312345):""")
  historico_filtrado = {}
  for chave, valor in historico.items():
      if valor['status'] == 'ativo':
        historico_filtrado[chave] = valor
  print(historico_filtrado) # Rapha - aqui precisa deixar mais bonito de ler também?

  id = input("\nInforme o id do registro que deve ser DELETADO do histórico (os id's estão no seu histórico acima): ")

  if id not in historico.keys():
    print("\nA chave informada não consta do seu histórico.\n")
    chat_pagina2()

  elif historico[id]["status"] == 'deletado':
    print("\nVocê já havia deletado esse registro.\n")
    chat_pagina2()

  else:
    historico[id]["status"] = 'deletado'
    print("\nRegistro deletado com sucesso!\n")
    report(historico)
    chat_pagina2()

def soma_receitas(): # Daniel - puxa a soma das receitas
  with open('financas_historico.json','r') as file:
    conteudo = file.read()
    if len(conteudo) == 0:
      historico = {}
    else:
      historico = json.loads(conteudo)
  historico_filtrado = {}
  for chave, valor in historico.items():
      if valor['status'] == 'ativo':
        historico_filtrado[chave] = valor
  soma_receitas = 0
  for chave, valor in historico_filtrado.items():
      if valor['movimentacao'] == 'receita':
          soma_receitas += valor['valor']
  # print(f'O seu total de receitas é R$ {soma_receitas}')
  # chat_pagina4()
  return soma_receitas
  # Adam - retornar valor

def soma_despesas(): # Daniel - puxa a soma das despesas
  with open('financas_historico.json','r') as file:
    conteudo = file.read()
    if len(conteudo) == 0:
      historico = {}
    else:
      historico = json.loads(conteudo)
  historico_filtrado = {}
  for chave, valor in historico.items():
      if valor['status'] == 'ativo':
        historico_filtrado[chave] = valor
  soma_despesas = 0
  for chave, valor in historico_filtrado.items():
      if valor['movimentacao'] == 'despesa':
          soma_despesas += valor['valor']
  soma_despesas *= -1
  # print(f'O seu total de despesas é R$ {soma_despesas}')
  # chat_pagina4()
  return soma_despesas
  # Adam - retornar valor (negativo?)

def get_entry(): # gaby acrescentei a opcao 3 get_entry
    historico = get_history()
    print("Histórico Completo:")
    for id, entry in historico.items():
        print(f"ID: {id}, Data: {entry['data']}, Tipo: {entry['movimentacao']}, Valor: {entry['valor']}") #olha dentro de cada chave com o for e exibi o valor
    chat_pagina2() # mostrar o valor da taxa se for investimento // não está mostrando agora

def get_history():
  with open('financas_historico.json','r') as file:
    conteudo = file.read()
    if len(conteudo) == 0:
      historico = {}
    else:
      historico = json.loads(conteudo)
  return historico

def report(history):
  with open('financas_historico.json','wt') as file:
    json.dump(history, file, indent=4, sort_keys=True)

def get_report(): # Daniel (está ok, exibindo apenas os "ativos")
  with open('financas_historico.json','r') as file:
    conteudo = file.read()
    if len(conteudo) == 0:
      historico = {}
    else:
      historico = json.loads(conteudo)
  print("""Abaixo está o seu histórico de lançamentos.
  Copie o id correspondente caso deseje atualizar/deletar algum registro (ex. 202312345):""")
  historico_filtrado = {}
  for chave, valor in historico.items():
      if valor['status'] == 'ativo':
        historico_filtrado[chave] = valor
  print(json.dumps(historico_filtrado, indent = 4)) # Adam - deixar a saída mais amigavel, i.e, mais facil de ler ## Rapha - Corrigi aqui
  chat_pagina2()

def new_income():
  valor = float(input("Informe o valor: "))
  historico = get_history()
  info = {
      make_id(): {
      "status": "ativo",
      "movimentacao": "receita",
      "valor": valor,
      "data": datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M:%S.%f"),
      "dia": datetime.datetime.now().day,
      "mes": datetime.datetime.now().month,
      "ano": datetime.datetime.now().year
    }
  }
  historico.update(info)
  report(historico)
  print(f'A receita de R$ {valor} foi adicionado com sucesso!\n')
  chat_pagina3()

def new_expense():
  valor = float(input("Informe o valor: "))
  historico = get_history()
  info = {
    make_id(): {
      "status": "ativo",
      "movimentacao": "despesa",
      "valor": (-1)*valor,
      "data": datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M:%S.%f"),
      "dia": datetime.datetime.now().day,
      "mes": datetime.datetime.now().month,
      "ano": datetime.datetime.now().year
    }
  }

  historico.update(info)
  report(historico)
  print(f'A despesa de R$ {valor} foi adicionado com sucesso!\n')
  chat_pagina3()

def new_investiment():
  valor = float(input("Informe o valor: "))
  taxa = float(input("Informe a taxa diária de rendimento (em porcentagem): "))
  historico = get_history()
  info = {
    make_id(): {
      "status": "ativo",
      "movimentacao": "investimento",
      "valor": valor,
      "juros": taxa,
      "data": datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M:%S.%f"),
      "dia": datetime.datetime.now().day,
      "mes": datetime.datetime.now().month,
      "ano": datetime.datetime.now().year
    }
  }

  historico.update(info)
  report(historico)
  print(f'O investimento de R$ {valor} sob taxa de {taxa}% foi adicionado com sucesso!\n')
  chat_pagina3()

def make_id():
  date = datetime.datetime.strftime(datetime.datetime.now(),"%Y,%m,%d,%H,%M,%S,%f")
  date = list(date.split(sep=","))

  digito_1 = reduce(lambda x,y: int(x)+int(y),date) % 10
  digito_2 = reduce(lambda x,y: int(x)**2+int(y)**2,date) % 10

  concat_value = digito_1*10 + digito_2
  id = date[:3]
  id.append(str(concat_value))
  id = ''.join(id)
  return id

def chat_pagina1(): # gaby acrescentei a opcao 3
    opcao = int(input("""
    O que deseja fazer:
    1: Criar novo histórico
    2: Atualizar histórico existente
    3: Mostrar o histórico completo
    0: Encerrar

    Opção: """))

    if opcao not in (1, 2, 3, 0):  # gaby acrescentei a opcao 3
        print("Opção inválida.\n\n")
        chat_pagina1()

    acao = {
        1: new_file,
        2: chat_pagina2,
        3: get_entry, # gaby acrescentei a opcao 3
        0: encerrar_programa
    }
    acao[opcao]()

def chat_pagina2():
  opcao = int(input("""
  O que deseja fazer?
    1: Nova movimentação
    2: Atualizar movimentação
    3: Deletar movimentação
    4: Ver movimentações
    5: Gerar relatório
    6: Voltar ao menu anterior
    0: Encerrar

    Opção: """))

  if opcao not in (1,2,3,4,5,6,0):
    print("Opção invalida.\n\n")
    chat_pagina2()

  acao = {
      1: chat_pagina3,
      2: update_entry, # update_receita; update_despesas; update_investimento
      3: delete_entry, # get_history() -> update['status'] -> 'deleted'
      4: get_entry, # ver as movimentações (get_history())
      5: chat_pagina4, #get_report
      6: chat_pagina1,
      0: encerrar_programa
  }
  acao[opcao]()

def chat_pagina3():
  opcao = int(input("""Digite o tipo de movimentação:
  1: Receita
  2: Despesa
  3: Investimento
  4: Voltar ao menu anterior
  0: Encerrar

  Opção: """))

  if opcao not in (1,2,3,4,0):
    print("Opção inválida.\n\n")
    chat_pagina3()

  acao = {
      1: new_income,
      2: new_expense,
      3: new_investiment,
      4: chat_pagina2,
      0: encerrar_programa
  }
  acao[opcao]()

def chat_pagina4():
  opcao = int(input("""Digite o tipo de movimentação:
  1: Histórico de lançamentos
  2: Soma das receitas
  3: Soma das despesas
  4: Soma dos investimentos
  5: Saldo futuro
  6: Voltar ao menu anterior
  0: Encerrar

  Opção: """))

  if opcao not in (1,2,3,4,5,6,0):
    print("Opção inválida.\n\n")
    chat_pagina4()

  acao = {
      1: get_report,
      2: mostra_soma_receitas,
      3: mostra_soma_despesas,
      4: mostra_soma_investimentos,
      5: valores_futuros,
      6: chat_pagina2,
      0: encerrar_programa
  }
  acao[opcao]()

def calcular_dias(dia, mes, ano, dia_futuro, mes_futuro, ano_futuro):

  data_inicio = datetime.datetime(ano, mes, dia)
  data_fim = datetime.datetime(ano_futuro, mes_futuro, dia_futuro)
  tempo = (data_fim - data_inicio).days ##.days converte a data em um inteiro
  return tempo

def calcular_montante(valor, taxa, tempo):
  montante = valor * (1 + taxa/100) ** tempo
  return montante

def valores_futuros():
  receitas = soma_receitas()
  despesa = soma_despesas()
  investimento, tempo = soma_investimentos()
  print(f'A soma dos valores futuros ao longo de {tempo} dias é: R${(receitas + investimento - despesa):.2f}')
  chat_pagina4()

def get_investimentos():
  with open('financas_historico.json','r') as file:
    conteudo = file.read()
    if len(conteudo) == 0:
      historico = {}
    else:
      historico = json.loads(conteudo)
  historico_filtrado = {}
  for chave, valor in historico.items():
    if valor['status'] == 'ativo' and valor['movimentacao'] == 'investimento':
      historico_filtrado[chave] = valor
  return historico_filtrado

def soma_investimentos():
  dia_futuro, mes_futuro, ano_futuro = map(lambda x: int(x), input("\nInforme a data que deseja realizar o cálculo do montante (Exemplo: 20/01/2020): ").split(sep = "/"))
  historico_filtrado = get_investimentos()

  investimento = []
  for item in historico_filtrado.keys():
    tempo = calcular_dias(int(historico_filtrado[item]['dia']), int(historico_filtrado[item]['mes']), int(historico_filtrado[item]['ano']), dia_futuro, mes_futuro, ano_futuro)
    calcular = calcular_montante(float(historico_filtrado[item]['valor']), float(historico_filtrado[item]['juros']), tempo)
    investimento.append(calcular)
  # print(f'O valor investido depois de {tempo} dias é R${sum(investimento):.2f}!')
  # chat_pagina4()
  return sum(investimento), tempo

def mostra_soma_receitas():
  mostrar_receita = soma_receitas()
  print(f'A soma de receitas é: R${mostrar_receita}')
  chat_pagina4()

def mostra_soma_despesas():
  mostrar_despesa = soma_despesas()
  print(f'A soma de despesas é: R${mostrar_despesa}')
  chat_pagina4()

def mostra_soma_investimentos():
  mostrar_investimentos, tempo = soma_investimentos()
  print(f'A soma de investimentos ao longo de {tempo} dias é: R${mostrar_investimentos:.2f}')
  chat_pagina4()

def encerrar_programa():
  print("Programa encerrado pelo usuário")
  print("Conte comigo na luta contra o Leão!")

def main():
  chat_pagina1()

In [ ]:
main()


    O que deseja fazer:
    1: Criar novo histórico
    2: Atualizar histórico existente
    3: Mostrar o histórico completo
    0: Encerrar

    Opção: 2

  O que deseja fazer?
    1: Nova movimentação
    2: Atualizar movimentação
    3: Deletar movimentação
    4: Ver movimentações
    5: Gerar relatório
    6: Voltar ao menu anterior
    0: Encerrar

    Opção: 5
Digite o tipo de movimentação:
  1: Histórico de lançamentos
  2: Soma das receitas
  3: Soma das despesas
  4: Soma dos investimentos
  5: Saldo futuro
  6: Voltar ao menu anterior
  0: Encerrar

  Opção: 4

Informe a data que deseja realizar o cálculo do montante (Exemplo: 20/01/2020): 20/10/2024
A soma de investimentos ao longo de 334 dias é: R$214512.39
Digite o tipo de movimentação:
  1: Histórico de lançamentos
  2: Soma das receitas
  3: Soma das despesas
  4: Soma dos investimentos
  5: Saldo futuro
  6: Voltar ao menu anterior
  0: Encerrar

  Opção: 5

Informe a data que deseja realizar o cálculo do montante (Ex